In [ ]:
!pip install tensorflow-addons

In [ ]:
import os
import glob
import random
import numpy as np
import pandas as pd

import tensorflow_addons as tfa
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm

from PIL import Image

from tensorflow.keras.utils import to_categorical

import seaborn as sns
import matplotlib.image as img
import matplotlib.pyplot as plt

In [ ]:
#To check if i am working on my gpu
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)
print(tf.config.list_physical_devices('GPU'))

In [ ]:
#load the drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
file_Training_path = '/content/drive/MyDrive/humanactionreco/Training_set.csv'
train_csv = pd.read_csv(file_Training_path)
train_csv.head()

In [ ]:
file_Testing_path = '/content/drive/MyDrive/humanactionreco/Testing_set.csv'
test_csv = pd.read_csv(file_Testing_path)
test_csv.head()

In [ ]:
train_fol = glob.glob("/content/drive/MyDrive/humanactionreco/train/*") 
test_fol = glob.glob("/content/drive/MyDrive/humanactionreco/test/*")

In [ ]:
train_csv

In [ ]:
train_csv.label.value_counts()

In [ ]:
import plotly.express as px
l = train_csv.label.value_counts()
fig = px.pie(train_csv, values=l.values, names=l.index, title='Distribution of Human Activity')
fig.show()

In [ ]:
filename = train_csv['filename']
situation = train_csv['label']

In [ ]:
filename

In [ ]:
situation

In [ ]:
import os
import random
from PIL import Image
import matplotlib.pyplot as plt

train_dir = "/content/drive/MyDrive/humanactionreco/train/"
train_csv_path = "/content/drive/MyDrive/humanactionreco/Training_set.csv"

# Load the train CSV file
train_csv = pd.read_csv(train_csv_path)

def disp():
    num = random.randint(1,100)
    imgg = "Image_{}.jpg".format(num)
    img_path = os.path.join(train_dir, imgg)
    if os.path.exists(img_path):
        try:
            testImage = Image.open(img_path)
            plt.imshow(testImage)
            label = train_csv.loc[train_csv['filename'] == imgg, 'label'].item()
            plt.title("{}".format(label))
            print("Opened image file: {}".format(img_path))
        except Exception as e:
            print("Error opening image file: {}".format(e))
    else:
        print("File not found: {}".format(img_path))

# Test the function
disp()

Processing Data

In [ ]:
train_fol = ['/content/drive/MyDrive/humanactionreco/train/'] # list of paths to training directories

# Initialize empty lists to hold image data and labels
img_data = []
img_label = []

# Loop through each training directory and load images
for fol in train_fol:
    for filename in os.listdir(fol):
        # Load the image and resize it
        img_path = os.path.join(fol, filename)
        img = Image.open(img_path)
        img = img.resize((160, 160))
        # Convert the image to a numpy array and add it to img_data
        img_data.append(np.array(img))
        # Get the label for the image and add it to img_label
        if filename[0].isdigit():
           label = situation[int(filename[0])-1]
        else:
           label = None # or some default label
        img_label.append(label)


In [ ]:
inp_shape = (160,160,3)

In [ ]:
y_train = to_categorical(np.asarray(train_csv['label'].factorize()[0]))
print(y_train[0])

In [ ]:
print(len(y_train))

In [ ]:
iii = img_data
iii = np.asarray(iii)
type(iii)

In [ ]:
iii = iii[:len(y_train)]

In [ ]:
vgg_model = Sequential()

pretrained_model= tf.keras.applications.VGG16(include_top=False,
                   input_shape=(160,160,3),
                   pooling='avg',classes=15,
                   weights='imagenet')

for layer in pretrained_model.layers:
        layer.trainable=False

vgg_model.add(pretrained_model)
vgg_model.add(Flatten())
vgg_model.add(Dense(512, activation='relu'))
vgg_model.add(Dense(15, activation='softmax'))

In [ ]:
vgg_model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
vgg_model.summary()

In [ ]:
history = vgg_model.fit(iii,y_train, epochs=60)

In [ ]:
losss = history.history['loss']
plt.plot(losss)

In [ ]:
accu = history.history['accuracy']
plt.plot(accu)

In [ ]:
#testing
# Function to read images as array

def read_image(fn):
    image = Image.open(fn)
    return np.asarray(image.resize((160,160)))

In [ ]:
!pip install opencv-python-headless

In [ ]:
import cv2

In [ ]:
def test_predict(test_image):
    result = vgg_model.predict(np.asarray([read_image(test_image)]))
    predicted_class = np.argmax(result)
    predicted_class_name = train_csv.label.unique()[predicted_class]
    probability = np.max(result) * 100
    
    print(f"Probability: {probability:.2f}%\nPredicted class: {predicted_class_name}")
    
    image = cv2.imread(test_image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(image)


In [ ]:
test_predict('/content/drive/MyDrive/humanactionreco/test/Image_4.jpg')